In [22]:
fabric_endpoint = "somethingsecret.datawarehouse.pbidedicated.windows.net"
fabric_dwh_name = "the name of your data warehouse"

In [23]:
conn_str = "DRIVER={ODBC Driver 18 for SQL Server};SERVER=" + fabric_endpoint +",1433;Database=" + fabric_dwh_name +";encrypt=Yes;TrustServerCertificate=Yes"
print(conn_str)

DRIVER={ODBC Driver 18 for SQL Server};SERVER=somethingsecret.datawarehouse.pbidedicated.windows.net,1433;Database=the name of your data warehouse;encrypt=Yes;TrustServerCertificate=Yes


In [20]:
from azure.identity import AzureCliCredential
import struct
from itertools import chain, repeat

credential = AzureCliCredential()
token = credential.get_token("https://database.windows.net//.default")
token_value = bytes(token.token, "UTF-8")
encoded_bytes = bytes(chain.from_iterable(zip(token_value, repeat(0))))
token_bytes = struct.pack("<i", len(encoded_bytes)) + encoded_bytes
attrs_before = {1256: token_bytes}

In [10]:
import pyodbc

connection = pyodbc.connect(
                conn_str,
                attrs_before=attrs_before,
                autocommit=True,
            )

In [ ]:
def print_rows():
    print(connection.cursor().execute("select * from repro_dt2_issue_schema.repro_dt2_issue_table").fetchall())

In [11]:
connection.cursor().execute("create schema repro_dt2_issue_schema")
connection.cursor().execute("create table repro_dt2_issue_schema.repro_dt2_issue_table (id int, name varchar(100), dt2 datetime2(6))")
print_rows()

In [14]:
connection.cursor().execute("insert into repro_dt2_issue_schema.repro_dt2_issue_table values (1, 'test', '2021-01-01 00:00:00.000000')")
print_rows()

In [16]:
sql = "insert into repro_dt2_issue_schema.repro_dt2_issue_table(id, name, dt2) values (?, ?, ?)"
bindings = [2, 'another test', '2021-01-02 00:00:00.000000']
connection.cursor().execute(sql, bindings)
print_rows()

all of the above works fine

it's only when we are working with `datetime` objects that we get the error

In [19]:
import datetime
from decimal import Decimal


sql = "insert into repro_dt2_issue_schema.repro_dt2_issue_table(id, name, dt2) values (?, ?, ?)"
bindings = [Decimal(3), 'another test', datetime.datetime(2021, 1, 3, 0, 0)]
connection.cursor().execute(sql, bindings)
print_rows()

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]A precision value must be specified. The range of valid values is an integer value between 0 and 6. (24598) (SQLExecDirectW)')

the error:

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]A precision value must be specified. The range of valid values is an integer value between 0 and 6. (24598) (SQLExecDirectW)')